In [1]:
import os
import io
import struct
from typing import List, Tuple, Optional
from enum import Enum
import pandas as pd

from blowfish import Blowfish

In [2]:
class PackEntryType(Enum):
  Nop = 0
  Folder = 1
  File = 2

In [3]:
def generate_final_blowfish_key(password: str, salt: bytes) -> bytes:
    """
    Reproduces the 'GenerateFinalBlowfishKey' logic in C#.
    """
    # 1) Limit key length to max of 56
    plain_key_length = min(len(password), 56)

    # 2) Convert password to ASCII bytes
    a_key = password.encode("ascii")

    # 3) Create a 56-byte base key buffer
    b_key = bytearray(56)
    
    # Copy salt into b_key
    # Equivalent to `Array.ConstrainedCopy(salt, 0, bKey, 0, salt.Length)`
    b_key[:len(salt)] = salt

    # 4) Generate the final blowfish key by XOR-ing
    #    the ASCII password bytes with the corresponding part of b_key.
    bf_key = bytearray(plain_key_length)
    for x in range(plain_key_length):
        bf_key[x] = a_key[x] ^ b_key[x]

    return bytes(bf_key)

In [4]:
def read_string_with_length(stream, byte_count: int) -> str:
    """
    Read 'byte_count' bytes, then decode using self._encoding,
    stopping at the first 0 (null terminator) if present.
    """
    buffer = stream.read(byte_count)

    # Find null terminator offset (if any)
    terminator_offset = byte_count
    for i in range(byte_count):
        if buffer[i] == 0:
            terminator_offset = i
            break

    return buffer[:terminator_offset].decode('ascii', errors='replace')

In [5]:
data_pk2_path = os.path.join(os.getcwd(), 'data', 'Data.pk2')
media_pk2_path = os.path.join(os.getcwd(), 'data', 'Media.pk2')
print(data_pk2_path)
print(media_pk2_path)

c:\Users\htdun\Desktop\workspace\pk2-extractor\data\Data.pk2
c:\Users\htdun\Desktop\workspace\pk2-extractor\data\Media.pk2


In [6]:
pk2key = '169841'
salt = [0x03, 0xF8, 0xE4, 0x44, 0x88, 0x99, 0x3F, 0x64, 0xFE, 0x35]

In [7]:
key = generate_final_blowfish_key(pk2key, bytes(salt))
key

b'2\xce\xdd|\xbc\xa8'

In [8]:
blowfish = Blowfish()
blowfish.Initialize(key)

In [9]:
_media_file_stream = open(media_pk2_path, 'rb')

In [10]:
signature = _media_file_stream.read(30)
version =  struct.unpack('<i', _media_file_stream.read(4))[0]
encrypted = _media_file_stream.read(1)
encryption_checksum = _media_file_stream.read(16)
payload = _media_file_stream.read(205)


In [11]:
Header = {
    'signature': signature,
    'version': version,
    'encrypted': encrypted,
    'encryption_checksum': encryption_checksum,
    'payload': payload
}

In [12]:
blowfish_checksum_decoded = "Joymax Pak File"

In [13]:
if blowfish and encrypted == b'\x01':
    temp_checksum = blowfish.Encode(blowfish_checksum_decoded.encode('ascii'))
    if temp_checksum is None or temp_checksum[0] != encryption_checksum[0] or temp_checksum[1] != encryption_checksum[1] or temp_checksum[2] != encryption_checksum[2]:
        raise Exception('Failed to open JoymaxPackFile: The password or salt is wrong.')

In [14]:
def read_block_at(position: int):
    _media_file_stream.seek(position, io.SEEK_SET)
    buffer = _media_file_stream.read(128 * 20)
    if blowfish is not None:
      entry_buffer = io.BytesIO(blowfish.Decode(buffer))
    else:
      entry_buffer = io.BytesIO(buffer)

    entries = []
    for _ in range(20):
      entry = {
        "Type": PackEntryType(entry_buffer.read(1)[0]),
        "Name": read_string_with_length(entry_buffer, 89).rstrip('\0'),
        "CreationTime": struct.unpack('<q', entry_buffer.read(8))[0],
        "ModifyTime": struct.unpack('<q', entry_buffer.read(8))[0],
        "DataPosition": struct.unpack('<q', entry_buffer.read(8))[0],
        "DataSize": struct.unpack('<i', entry_buffer.read(4))[0],
        "NextBlock": struct.unpack('<q', entry_buffer.read(8))[0],
        "Payload": entry_buffer.read(2)
      }
      entries.append(entry)
      # print(entry)

    return {
      "Position": position,
      "Entries": entries
    }


def read_blocks_at(position: int):
    result = []

    block = read_block_at(position)
    result.append(block)

    if block["Entries"][19]["NextBlock"] > 0:
        result.extend(read_blocks_at(block["Entries"][19]["NextBlock"]))

    return result

In [15]:
Root = read_blocks_at(256)

In [16]:
blocks_in_memory = {
  "": Root
}

In [17]:
blocks_in_memory

{'': [{'Position': 256,
   'Entries': [{'Type': <PackEntryType.Folder: 1>,
     'Name': '.',
     'CreationTime': 133336205048819447,
     'ModifyTime': 133336205048819447,
     'DataPosition': 256,
     'DataSize': 0,
     'NextBlock': 0,
     'Payload': b'\x00\x00'},
    {'Type': <PackEntryType.Folder: 1>,
     'Name': 'acobject',
     'CreationTime': 133336205048829450,
     'ModifyTime': 133336205048829450,
     'DataPosition': 2816,
     'DataSize': 0,
     'NextBlock': 0,
     'Payload': b'\x00\x00'},
    {'Type': <PackEntryType.Folder: 1>,
     'Name': 'config',
     'CreationTime': 133336205049179438,
     'ModifyTime': 133336205049179438,
     'DataPosition': 35136,
     'DataSize': 0,
     'NextBlock': 0,
     'Payload': b'\x00\x00'},
    {'Type': <PackEntryType.Folder: 1>,
     'Name': 'Effect',
     'CreationTime': 133336205049679437,
     'ModifyTime': 133336205049679437,
     'DataPosition': 41896,
     'DataSize': 0,
     'NextBlock': 0,
     'Payload': b'\x00\x00'},
   

In [18]:
server_dep = os.path.join('server_dep', 'silkroad', 'textdata')
server_dep

'server_dep\\silkroad\\textdata'

In [19]:
item_text_data = dict()

## TextUISystem.txt

In [20]:
item_data = os.path.join(server_dep, 'ItemData.txt')
item_data

'server_dep\\silkroad\\textdata\\ItemData.txt'

In [21]:
def get_entry_buffer(file_path: str) -> Optional[dict]:
  parent_folder_path = os.path.dirname(file_path)
  file_name = os.path.basename(file_path)
  if parent_folder_path not in blocks_in_memory:
    paths = parent_folder_path.split(os.path.sep)
    blocks = blocks_in_memory[""]
    current_path = ""

    for sub_folder_name in paths:
        for block in blocks:
            entries = block["Entries"]
            entry = next((e for e in entries if e["Name"] == sub_folder_name and e["Type"] == PackEntryType.Folder), None)
            if entry is None:
                continue
            
            current_path = os.path.join(current_path, entry["Name"])
            if current_path in blocks_in_memory:
                blocks = blocks_in_memory[current_path]
                break
            
            blocks = read_blocks_at(entry["DataPosition"])
            blocks_in_memory[current_path] = blocks
            break
        
  root = blocks_in_memory[parent_folder_path]
  entry = None
  for entries in root:
    entry = next((x for x in entries["Entries"] if x["Name"] == file_name.lower()), None)
    if entry:
      break

  _media_file_stream.seek(entry["DataPosition"], io.SEEK_SET)
  # _media_file_stream.read(entry["DataSize"])
  buffer = io.BytesIO(_media_file_stream.read(entry["DataSize"])) 

  return buffer
   

def get_lines(file_path: str) -> List[Tuple[str, int]]:

  buffer = get_entry_buffer(file_path)

  text = buffer.read().decode('utf-16', errors="replace")
  lines = text.split('\r\n')
  return lines  

def convert_string(value):
    try:
        # First, try to convert to an integer
        return int(value)
    except ValueError:
        try:
            # If it fails, try to convert to a float
            return float(value)
        except ValueError:
            # If both conversions fail, return the original string
            return value


In [22]:
item_data_files = get_lines(item_data)
item_data_files

['ItemData_100.txt',
 'ItemData_1000.txt',
 'ItemData_10150.txt',
 'ItemData_10300.txt',
 'ItemData_10350.txt',
 'ItemData_10400.txt',
 'ItemData_10450.txt',
 'ItemData_1050.txt',
 'ItemData_10500.txt',
 'ItemData_10550.txt',
 'ItemData_10600.txt',
 'ItemData_10650.txt',
 'ItemData_10700.txt',
 'ItemData_10750.txt',
 'ItemData_10800.txt',
 'ItemData_10850.txt',
 'ItemData_10900.txt',
 'ItemData_10950.txt',
 'ItemData_1100.txt',
 'ItemData_11000.txt',
 'ItemData_11050.txt',
 'ItemData_11100.txt',
 'ItemData_11150.txt',
 'ItemData_11200.txt',
 'ItemData_11250.txt',
 'ItemData_11300.txt',
 'ItemData_11350.txt',
 'ItemData_11400.txt',
 'ItemData_11450.txt',
 'ItemData_1150.txt',
 'ItemData_11500.txt',
 'ItemData_11550.txt',
 'ItemData_11600.txt',
 'ItemData_11650.txt',
 'ItemData_11700.txt',
 'ItemData_11750.txt',
 'ItemData_11800.txt',
 'ItemData_11850.txt',
 'ItemData_11900.txt',
 'ItemData_11950.txt',
 'ItemData_1200.txt',
 'ItemData_12000.txt',
 'ItemData_12050.txt',
 'ItemData_12100.t

In [23]:
for xxx in item_data_files:
  file_name = os.path.join(server_dep, xxx)
  lines = get_lines(file_name)
  for idx, line in enumerate(lines):
    values = line.split('\t')
    if len(values) < 11:
      continue
    Service = convert_string(values[0])
    ID = convert_string(values[1])
    CodeName = convert_string(values[2])
    ObjName = convert_string(values[3])
    NameStrId = convert_string(values[5])
    CashItem = convert_string(values[7])
    Bionic = convert_string(values[8])
    TypeId1 = convert_string(values[9])
    TypeId2 = convert_string(values[10])
    TypeId3 = convert_string(values[11])
    TypeId4 = convert_string(values[12])
    Country = convert_string(values[14])
    Rarity = convert_string(values[15])
    CanDrop = convert_string(values[20])
    CanUSe = convert_string(values[24])
    RequestLevelType1 = convert_string(values[32])
    RequestLevel1 = convert_string(values[33])
    RequestLevelType2 = convert_string(values[34])
    RequestLevel2 = convert_string(values[35])
    RequestLevelType3 = convert_string(values[36])
    RequestLevel3 = convert_string(values[37])
    RequestLevelType4 = convert_string(values[38])
    RequestLevel4 = convert_string(values[39])
    Speed1 = convert_string(values[46])
    Speed2 = convert_string(values[47])
    AssicFileIcon = convert_string(values[54])

    MaxStack = convert_string(values[57])
    ReqGender = convert_string(values[58])
    ReqStr = convert_string(values[59])
    ReqInt = convert_string(values[60])
    ItemClass = convert_string(values[61])
    Quivered = convert_string(values[86])
    SpeedClass = convert_string(values[92])
    TwoHanded = convert_string(values[93])
    Range = convert_string(values[94])
    Param1 = convert_string(values[66])
    Desc1 = convert_string(values[67])
    Param2 = convert_string(values[66])
    Desc2 = convert_string(values[67])
    Param3 = convert_string(values[66])
    Desc3 = convert_string(values[67])
    Param4 = convert_string(values[66])
    Desc4 = convert_string(values[67])

    IsStackable = TypeId2 == 3
    IsGold = IsStackable and TypeId3 == 5 and TypeId4 == 0
    IsEquip = TypeId2 == 1
    IsAvatar = IsEquip and TypeId3 in [13, 14]
    IsJobEquip = TypeId2 == 4
    IsFellowEquip = TypeId2 == 5
    IsJobOutfit = IsEquip and TypeId3 == 7 and TypeId4 in [4, 5]
    IsTrading = IsStackable and TypeId3 == 8
    IsSpecialtyGoodBox = IsTrading and TypeId4 == 3
    IsQuest = IsStackable and TypeId3 == 9
    IsAmmunition = IsStackable and TypeId3 == 4
    IsPet = TypeId2 == 2 and TypeId3 == 1
    IsGrowthPet = IsPet and TypeId4 == 1
    IsGrabPet = IsPet and TypeId4 == 2
    IsFellowPet = IsPet and TypeId4 == 3
    IsTransmonster = TypeId2 == 2 and TypeId3 == 2
    IsMagicCube = TypeId2 == 2 and TypeId3 == 3
    IsSkill = IsStackable and TypeId3 == 13 and TypeId4 == 1
    IsPotion = IsStackable and TypeId3 == 1
    IsHpPotion = IsPotion and TypeId4 == 1
    IsMpPotion = IsPotion and TypeId4 == 2
    IsAllPotion = IsPotion and TypeId4 == 3
    IsUniversalPill = IsStackable and TypeId3 == 2 and TypeId4 == 6
    IsPurificationPill = IsStackable and TypeId3 == 2 and TypeId4 == 1
    IsAbnormalPotion = IsStackable and TypeId3 == 2 and TypeId4 == 9
    IsCosHpPotion = IsPotion and TypeId4 == 4 and Param2 == 0
    IsFellowHpPotion = IsPotion and TypeId4 == 4 and Param2 != 0
    IsCosRevivalPotion = IsPotion and TypeId4 == 6
    IsHwanPotion = IsPotion and TypeId4 == 8
    IsHgpPotion = IsPotion and TypeId4 == 9 and Param1 == 10
    IsPet2SatietyPotion = IsPotion and TypeId4 == 9 and Param1 > 10
    IsRepairKit = IsPotion and TypeId4 == 10
    IsArmor = (IsEquip and TypeId3 == 1) or TypeId3 in [2, 3, 9, 10, 11]
    IsShield = IsEquip and TypeId3 == 4
    IsAccessory = (IsEquip and TypeId3 == 5) or TypeId3 == 12
    IsWeapon = IsEquip and TypeId3 == 6
    Degree = (int(ItemClass) - 1) / 3 + 1
    DegreeOffset = int(ItemClass) - 3 * ((int(ItemClass) - 1) / 3) - 1

    item_text_data[ID] = {
      "Service": Service,
      "ID": ID,
      "CodeName": CodeName,
      "ObjName": ObjName,
      "NameStrId": NameStrId,
      "CashItem": CashItem,
      "Bionic": Bionic,
      "TypeId1": TypeId1,
      "TypeId2": TypeId2,
      "TypeId3": TypeId3,
      "TypeId4": TypeId4,
      "Country": Country,
      "Rarity": Rarity,
      "CanDrop": CanDrop,
      "CanUSe": CanUSe,
      "RequestLevelType1": RequestLevelType1,
      "RequestLevel1": RequestLevel1,
      "RequestLevelType2": RequestLevelType2,
      "RequestLevel2": RequestLevel2,
      "RequestLevelType3": RequestLevelType3,
      "RequestLevel3": RequestLevel3,
      "RequestLevelType4": RequestLevelType4,
      "RequestLevel4": RequestLevel4,
      "Speed1": Speed1,
      "Speed2": Speed2,
      "AssicFileIcon": AssicFileIcon,
      "MaxStack": MaxStack,
      "ReqGender": ReqGender,
      "ReqStr": ReqStr,
      "ReqInt": ReqInt,
      "ItemClass": ItemClass,
      "Quivered": Quivered,
      "SpeedClass": SpeedClass,
      "TwoHanded": TwoHanded,
      "Range": Range,
      "Param1": Param1,
      "Desc1": Desc1,
      "Param2": Param2,
      "Desc2": Desc2,
      "Param3": Param3,
      "Desc3": Desc3,
      "Param4": Param4,
      "Desc4": Desc4,
      "IsStackable": IsStackable,
      "IsGold": IsGold,
      "IsEquip": IsEquip,
      "IsAvatar": IsAvatar,
      "IsJobEquip": IsJobEquip,
      "IsFellowEquip": IsFellowEquip,
      "IsJobOutfit": IsJobOutfit,
      "IsTrading": IsTrading,
      "IsSpecialtyGoodBox": IsSpecialtyGoodBox,
      "IsQuest": IsQuest,
      "IsAmmunition": IsAmmunition,
      "IsPet": IsPet,
      "IsGrowthPet": IsGrowthPet,
      "IsGrabPet": IsGrabPet,
      "IsFellowPet": IsFellowPet,
      "IsTransmonster": IsTransmonster,
      "IsMagicCube": IsMagicCube,
      "IsSkill": IsSkill,
      "IsPotion": IsPotion,
      "IsHpPotion": IsHpPotion,
      "IsMpPotion": IsMpPotion,
      "IsAllPotion": IsAllPotion,
      "IsUniversalPill": IsUniversalPill,
      "IsPurificationPill": IsPurificationPill,
      "IsAbnormalPotion": IsAbnormalPotion,
      "IsCosHpPotion": IsCosHpPotion,
      "IsFellowHpPotion": IsFellowHpPotion,
      "IsCosRevivalPotion": IsCosRevivalPotion,
      "IsHwanPotion": IsHwanPotion,
      "IsHgpPotion": IsHgpPotion,
      "IsPet2SatietyPotion": IsPet2SatietyPotion,
      "IsRepairKit": IsRepairKit,
      "IsArmor": IsArmor,
      "IsShield": IsShield,
      "IsAccessory": IsAccessory,
      "IsWeapon": IsWeapon,
      "Degree": Degree,
      "DegreeOffset": DegreeOffset
    }


In [24]:
item_text_data

{55: {'Service': 1,
  'ID': 55,
  'CodeName': 'ITEM_ETC_CURE_ALL_01',
  'ObjName': '?? ??(?)',
  'NameStrId': 'SN_ITEM_ETC_CURE_ALL_01',
  'CashItem': 0,
  'Bionic': 0,
  'TypeId1': 3,
  'TypeId2': 3,
  'TypeId3': 2,
  'TypeId4': 6,
  'Country': 3,
  'Rarity': 0,
  'CanDrop': 3,
  'CanUSe': 1,
  'RequestLevelType1': -1,
  'RequestLevel1': 0,
  'RequestLevelType2': -1,
  'RequestLevel2': 0,
  'RequestLevelType3': -1,
  'RequestLevel3': 0,
  'RequestLevelType4': -1,
  'RequestLevel4': 0,
  'Speed1': 0,
  'Speed2': 0,
  'AssicFileIcon': 'item\\etc\\cure_all_01.ddj',
  'MaxStack': 50,
  'ReqGender': 2,
  'ReqStr': 0,
  'ReqInt': 0,
  'ItemClass': 1,
  'Quivered': 0,
  'SpeedClass': 0,
  'TwoHanded': 0,
  'Range': 0,
  'Param1': 0.0,
  'Desc1': 0.0,
  'Param2': 0.0,
  'Desc2': 0.0,
  'Param3': 0.0,
  'Desc3': 0.0,
  'Param4': 0.0,
  'Desc4': 0.0,
  'IsStackable': True,
  'IsGold': False,
  'IsEquip': False,
  'IsAvatar': False,
  'IsJobEquip': False,
  'IsFellowEquip': False,
  'IsJobOutfit

In [25]:
df = pd.DataFrame.from_dict(item_text_data, orient='index')
df.reset_index(inplace=True)
df.rename(columns={'index': 'Key'}, inplace=True)

In [26]:
df

,Key,Service,ID,CodeName,ObjName,NameStrId,CashItem,Bionic,TypeId1,TypeId2,...,IsHwanPotion,IsHgpPotion,IsPet2SatietyPotion,IsRepairKit,IsArmor,IsShield,IsAccessory,IsWeapon,Degree,DegreeOffset
0,55,1,55,ITEM_ETC_CURE_ALL_01,?? ??(?),SN_ITEM_ETC_CURE_ALL_01,0,0,3,3,...,False,False,False,False,True,False,False,False,1.000000,0.0
1,56,1,56,ITEM_ETC_CURE_ALL_02,?? ??(?),SN_ITEM_ETC_CURE_ALL_02,0,0,3,3,...,False,False,False,False,True,False,False,False,1.333333,0.0
2,57,1,57,ITEM_ETC_CURE_ALL_03,?? ??(?),SN_ITEM_ETC_CURE_ALL_03,0,0,3,3,...,False,False,False,False,True,False,False,False,1.666667,0.0
3,58,1,58,ITEM_ETC_CURE_ALL_04,?? ?? ?? (?),SN_ITEM_ETC_CURE_ALL_04,0,0,3,3,...,False,False,False,False,True,False,False,False,2.000000,0.0
4,59,1,59,ITEM_ETC_CURE_ALL_05,?? ?? ?? (?),SN_ITEM_ETC_CURE_ALL_05,0,0,3,3,...,False,False,False,False,True,False,False,False,2.333333,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20684,945,1,945,ITEM_CH_M_CLOTHES_04_LA_B,?? ???? ??,SN_ITEM_CH_CLOTHES_04_LA_B,0,0,3,1,...,False,False,False,False,True,False,False,False,4.333333,0.0
20685,946,1,946,ITEM_CH_M_CLOTHES_04_LA_C,?? ???? ??,SN_ITEM_CH_CLOTHES_04_LA_C,0,0,3,1,...,False,False,False,False,True,False,False,False,4.666667,0.0
20686,947,1,947,ITEM_CH_M_CLOTHES_05_LA_A,?? ??? ??,SN_ITEM_CH_CLOTHES_05_LA_A,0,0,3,1,...,False,False,False,False,True,False,False,False,5.000000,0.0
20687,948,1,948,ITEM_CH_M_CLOTHES_05_LA_B,?? ??? ??,SN_ITEM_CH_CLOTHES_05_LA_B,0,0,3,1,...,False,False,False,False,True,False,False,False,5.333333,0.0


In [27]:
csv_file_path = 'item_data.csv'
df.to_csv(csv_file_path, index=False, encoding='utf-8')